In [1]:
import numpy as np
import pandas as pd
mv=pd.read_csv('movie_final.csv')

In [2]:
mv.head()

,id,title,adult,imdb_id,genres,poster_path,overview,vote_average,cast,crew,tags
0,862.0,Toy Story,NoAdult,tt0114709,"['Animation', 'Comedy', 'Family']",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"['Led', 'by', 'Woody,', ""Andy's"", 'toys', 'liv...",7.7,"['TomHanks', 'TimAllen', 'DonRickles', 'JimVar...",['JohnLasseter'],"led by woody, andy's toys live happily in his ..."
1,8844.0,Jumanji,NoAdult,tt0113497,"['Adventure', 'Fantasy', 'Family']",/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"['When', 'siblings', 'Judy', 'and', 'Peter', '...",6.9,"['RobinWilliams', 'JonathanHyde', 'KirstenDuns...",['JoeJohnston'],when siblings judy and peter discover an encha...
2,15602.0,Grumpier Old Men,NoAdult,tt0113228,"['Romance', 'Comedy']",/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"['A', 'family', 'wedding', 'reignites', 'the',...",6.5,"['WalterMatthau', 'JackLemmon', 'Ann-Margret',...",['HowardDeutch'],a family wedding reignites the ancient feud be...
3,31357.0,Waiting to Exhale,NoAdult,tt0114885,"['Comedy', 'Drama', 'Romance']",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"['Cheated', 'on,', 'mistreated', 'and', 'stepp...",6.1,"['WhitneyHouston', 'AngelaBassett', 'LorettaDe...",['ForestWhitaker'],"cheated on, mistreated and stepped on, the wom..."
4,11862.0,Father of the Bride Part II,NoAdult,tt0113041,['Comedy'],/e64sOI48hQXyru7naBFyssKFxVd.jpg,"['Just', 'when', 'George', 'Banks', 'has', 're...",5.7,"['SteveMartin', 'DianeKeaton', 'MartinShort', ...",['CharlesShyer'],just when george banks has recovered from his ...


In [3]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

def stemme(text):
    l=[]
    d=text.split()
    for i in d:
        l.append(ps.stem(i))
    return " ".join(l)

In [4]:
mv['tags']=mv['tags'].apply(stemme)
mv['tags'][0]

"led by woody, andy' toy live happili in hi room until andy' birthday bring buzz lightyear onto the scene. afraid of lose hi place in andy' heart, woodi plot against buzz. but when circumst separ buzz and woodi from their owner, the duo eventu learn to put asid their differences. tomhank timallen donrickl jimvarney wallaceshawn johnratzenberg anniepott johnmorri erikvondetten lauriemetcalf r.leeermey sarahfreeman pennjillett johnlasset noadult"

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

vectors=cv.fit_transform(mv['tags']).toarray()


In [6]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
similarmovie=cosine_similarity(vectors)

In [8]:
mv[mv['title'].str.contains('toy',na=False,case=False)==True].loc[0]

id                                                          862.0
title                                                   Toy Story
adult                                                     NoAdult
imdb_id                                                 tt0114709
genres                          ['Animation', 'Comedy', 'Family']
poster_path                      /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
overview        ['Led', 'by', 'Woody,', "Andy's", 'toys', 'liv...
vote_average                                                  7.7
cast            ['TomHanks', 'TimAllen', 'DonRickles', 'JimVar...
crew                                             ['JohnLasseter']
tags            led by woody, andy' toy live happili in hi roo...
Name: 0, dtype: object

In [9]:
mindex =mv[mv['title'].str.contains('Avatar',na=False,case=False)==True].index[0]
mv.loc[[mindex]]

matched=sorted(list(enumerate( similarmovie[mindex])),reverse=True,key=lambda x:x[1])[0:6]

for i in matched:
   c= mv.loc[i[0]]
   print(c.title)


Avatar
Avatar 2
Xtro 3: Watch the Skies
Alien Outpost
Bucky Larson: Born to Be a Star
Project Moon Base


In [10]:
def recomand(movie):
    #mindex =mv[mv.title.str.startwith(movie)].index[0]
    m=mv[mv['title'].str.contains(movie,na=False,case=False)==True]
    # print(m)
    output=[[]]
    originmatch=[["title","id"]]
    coun=0
    if len(m)>0:
      
        for iL in m.index:
            
            movievector=similarmovie[iL]
            matched=sorted(list(enumerate( movievector)),reverse=True,key=lambda x:x[1])[0:6]
            for i in matched:
                c= mv.loc[i[0]]
                if(c.vote_average.astype(float)>5.4):
                    output.append([c.title,i[0]])
       
    return output
       

In [11]:
result=recomand('Terminator')
# mv.iloc[25912]
for rs in result[1:]:
   g=(mv.iloc[rs[1]])
   print(g.title,g.poster_path,g.imdb_id,g.vote_average	)
    

Terminator 2: Judgment Day /2y4dmgWYRMYXdD1UyJVcn2HSd1D.jpg tt0103064 7.7
Terminator Genisys /5JU9ytZJyR3zmClGmVm9q4Geqbd.jpg tt1340138 5.8
Blind Justice /cWEdVUfd4bEqtnt5t9Is3J8otBs.jpg tt0005529 6.7
Terminator 3: Rise of the Machines /lz4xYdF1n09lyiCfZWtWT44SZiG.jpg tt0181852 5.9
The Kindred /i6CoZQgJzlX7TpBTobDUr6M08TY.jpg tt0091343 5.9
The Terminator /q8ffBuxQlYOHrvPniLgCbmKK4Lv.jpg tt0088247 7.4
Terminator 2: Judgment Day /2y4dmgWYRMYXdD1UyJVcn2HSd1D.jpg tt0103064 7.7
The Day of the Jackal /5dN5MIJGgJQ1yzgzQpxvhuZ2HN3.jpg tt0069947 7.3
Terminator Salvation /gw6JhlekZgtKUFlDTezq3j5JEPK.jpg tt0438488 5.9
The Exterminator /A0xIUPSH11W0LjlJU1Xk12z210q.jpg tt0080707 6.1
Big Jay Oakerson: Live at Webster Hall /oZo9AmzVVPZFWGV8yFlKFT7t4YF.jpg tt5814512 10.0
Three Wise Girls /58w9UGxOrC5Fh5GDUslpdmqZL92.jpg tt0023589 5.6
The Italian Connection /7pyJnQ4TL9SWbUGC0w7q5J9n8qc.jpg tt0068902 7.1
Terminator 3: Rise of the Machines /lz4xYdF1n09lyiCfZWtWT44SZiG.jpg tt0181852 5.9
Sunset Boulevard /

In [13]:
import pickle
pickle.dump(mv.to_dict(),open('movie_dict.pkl','wb'))
pickle.dump(similarmovie,open('similarity.pkl','wb'))